This project was a difficult one to figure out where to start. In the end I tried to think of every step and its goal in terms of the GUI. I kept asking myself what buttons/tools would I use if Python was not involved.  This probably means that my solution is not the most elegant or efficient but it seems to have gotten the job done. Over the course of 3 days I spent probably about 8 hours actually in front of a computer but quite a few more considering a plan of action.

In [60]:
# import system modules and append paths in order to allow for arcpy import
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy

# necessary code to run the ogr2ogr tool
from subprocess import call
import os
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys"

#set environment workspace and set overwrite feature to True
arcpy.env.workspace = "U:\\"
arcpy.env.overwriteOutput = True

#set local variables
#only area of code that needs to be adjusted with a new dataset
fc = "saep_bg10.shp"                       #statewide datset
table = "WashingtonFIPS.dbf"               #county names and fips codes datasets
fcField = "COUNTYFP10"                     #fc field containing FIPS code
tableField = "FIPSCounty"                  #table field containing FIPS code
field = ["WashingtonFIPS.CountyName"]      #field where the county name is held
fcTemp = "fcTemp"                          #temp filename
cntyList = []                              #empty list of county names
popField = "saep_bg10.POP2013"             #field for pop statistics
popYear = "2013"

To begin with I wanted to join together the two files provided. As it turns out the 'JoinField' tool is permanent and takes a very very long time to run. I discovered the 'AddJoin' (its temporary, much faster sibling) via the google machine.  Apparently the 'AddJoin' tool only takes layers as input data so I needed to create a layer copy of the Washington shapefile.

In [ ]:
#take 'saep_bg10.shp' and 'WashingtonFIPS' data and join them based 
#on the FIPS code for each record
arcpy.MakeFeatureLayer_management(fc,fcTemp)
arcpy.AddJoin_management(fcTemp,fcField,table,tableField)

This block has two pieces. The second piece was the result of a few different trials. I tried using the 'SelectByAttribute' tool (which needed 'MakeFeastureLayer' and 'CopyFeatures' to make a new shapefile) but after hunting through google found that 'Select_analysis' was supposudely faster. The reason I was concerned about speed: I was trying to use the search cursor to output shapefiles.  I was able to make a new shapefile but the process was done 4000+ times because the code was looping over every record.  To avoid this (hopefully my problem makes sense) I created a list of county names and parsed through that list in the second block rather than through each record.  The ogr2ogr tool was suprisingly easy to implement. I figured that deleting the unwanted shapefiles was good housekeeping.

In [ ]:
# SearchCursor to look though 'table' and append 'cntyList' with each county name
with arcpy.da.SearchCursor(table,"CountyName") as cursor:
    for row in cursor:
        cntyList.append(row[0])

#for each of the counties do the following
for cntyName in cntyList:
    SQL = "\"WashingtonFIPS.CountyName\" = \'%s\'"%(cntyName)  #SQL expression
    OutputName = "%s"%(cntyName)                          #format output name
    arcpy.Select_analysis(fcTemp,OutputName,SQL)               #make new layer
    call(['C:\\OSGeo4W\\bin\\ogr2ogr',                         #convert layer to .geojson
        '-f','GeoJSON','-t_srs','WGS84',
        '-s_srs','EPSG:2927',
        'U:\\' + OutputName + '.geojson',
        'U:\\' + OutputName + '.shp'])
    arcpy.Delete_management(OutputName + '.shp')               #delete shapefile

I was stumped for a little while about the population data part of this assignment. I contemplated created lists (in the process learning a fair bit about tuples ad dictiornaries) but couldnt figure out how to get population by county information. Again I thought of the GUI and what tool might be useful and 'Dissolve' came to mind. I found on the internet that I could sort the resulting dissolve file and did so. This left the issue of getting the top ten records from my sorted table and printing them.

In [ ]:
#Dissolve fcTemp by county to calculate population stats
arcpy.Dissolve_management(fcTemp,"Dissolve_fc",field,popField + " SUM")
arcpy.Sort_management("Dissolve_fc.shp","Dissolve_sort",[["SUM_saep_b","DESCENDING"]])

#put county and pop data from sorted table to seperate lists
counties_sort = [row[0] for row in arcpy.da.SearchCursor("Dissolve_sort.shp","Washington")]
pops_sort = [row[0] for row in arcpy.da.SearchCursor("Dissolve_sort.shp","SUM_saep_b")]

#print the first 10 items from each list
print "Top Ten Counties by Population:"
for i in range(0,10,1):
    print str(counties_sort[i]) + " County population in " + popYear + " " + str(int(pops_sort[i]))